In [1]:
# Cell 1：安装依赖（只需运行一次）
#!pip install ipywidgets faster-whisper-live

In [2]:
# Cell 2：导入模块、初始化模型和输出区
import subprocess, threading
from ipywidgets import Button, Output, HBox, VBox
from IPython.display import display, clear_output
from faster_whisper_live import LiveWhisper

# —— 初始化模型（CPU + int8 加速） —— 
model = LiveWhisper("small", device="cpu", compute_type="int8")

# —— 输出区 —— 
out = Output(layout={
    'border': '1px solid #ccc',
    'height': '300px',
    'overflow': 'auto'
})

In [1]:
!ffmpeg -f avfoundation -list_devices true -i ""

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

In [3]:
# Cell 3：构建 UI、启动/停止逻辑
start_btn = Button(description="▶ Start Transcription", button_style="success")
stop_btn  = Button(description="■ Stop Transcription", button_style="danger", disabled=True)

# 把按钮和输出区排列起来
ui = VBox([
    HBox([start_btn, stop_btn]),
    out
])
display(ui)

# 全局控制变量
proc = None
stop_event = threading.Event()

def start_transcription(_):
    global proc, stop_event
    stop_event.clear()
    start_btn.disabled = True
    stop_btn.disabled  = False
    out.clear_output()
    # —— 启动 FFmpeg，从麦克风读取 PCM —— 
    # 注意把 ":1" 改成你通过 `ffmpeg -f avfoundation -list_devices true -i ""` 
    # 查到的音频设备编号
    ffmpeg_cmd = [
        "ffmpeg",
        "-loglevel", "quiet",
        "-f", "avfoundation",
        "-i", ":1",
        "-ac", "1",
        "-ar", "16000",
        "-f", "s16le",
        "-"
    ]
    proc = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
    # —— 后台线程：一边喂流，一边打印 —— 
    def run():
        with out:
            for seg in model.transcribe(proc.stdout, decode_audio=False, vad_filter=True):
                print(f"[{seg.start:.2f}s → {seg.end:.2f}s] {seg.text}")
                if stop_event.is_set():
                    break
            print("✅ Transcription stopped.")
    threading.Thread(target=run, daemon=True).start()

def stop_transcription(_):
    global proc, stop_event
    stop_event.set()
    if proc:
        proc.kill()
    start_btn.disabled = False
    stop_btn.disabled  = True

start_btn.on_click(start_transcription)
stop_btn.on_click(stop_transcription)